<a href="https://colab.research.google.com/github/minjae0501/yolo_block/blob/master/lstm_yolo_detecting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
# !git clone https://github.com/minjae0501/yolo_block.git

Cloning into 'yolo_block'...
remote: Enumerating objects: 3012, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 3012 (delta 3), reused 13 (delta 1), pack-reused 2993
Receiving objects: 100% (3012/3012), 117.88 MiB | 50.70 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [15]:
# %pip install ultralytics
# %pip install mediapipe
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.227 🚀 Python-3.9.13 torch-1.12.1+cu116 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)
Setup complete ✅ (8 CPUs, 15.9 GB RAM, 207.1/232.3 GB disk)


In [16]:

import cv2
import mediapipe as mp
from ultralytics import YOLO
import numpy as np
import torch.nn as nn

In [17]:
import torch

if torch.cuda.is_available() == True:
    device = 'cuda:0'
else:
    device = 'cpu'

print(device)

cuda:0


In [18]:
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, seq_list):
        self.X = []
        self.y = []
        for dic in seq_list:
            self.y.append(dic['key'])
            self.X.append(dic['value'])
    def __getitem__(self, index):
        data = self.X[index]
        label = self.y[index]
        return torch.Tensor(np.array(data)), torch.tensor(np.array(int(label)))

    def __len__(self):
        return len(self.X)

In [19]:
class hand_LSTM(nn.Module):
    def __init__(self):
        super(hand_LSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size = 67, hidden_size = 128, num_layers=1, batch_first=True) # xyz 63 + x_center, y_center = 65
        self.lstm2 = nn.LSTM(input_size = 128, hidden_size = 256, num_layers=1, batch_first=True)
        # self.lstm3 = nn.LSTM(input_size = 256, hidden_size = 512, num_layers=1, batch_first=True)
        self.dropout1 = nn.Dropout(0.1)
        # self.lstm4 = nn.LSTM(input_size = 512, hidden_size = 256, num_layers=1, batch_first=True)
        self.lstm5 = nn.LSTM(input_size = 256, hidden_size = 128, num_layers=1, batch_first=True)
        self.lstm6 = nn.LSTM(input_size = 128, hidden_size = 64, num_layers=1, batch_first=True)
        self.dropout2 = nn.Dropout(0.1)
        self.lstm7 = nn.LSTM(input_size = 64, hidden_size = 32, num_layers=1, batch_first=True)
        self.fc = nn.Linear(32, 2)
    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        # x, _ = self.lstm3(x)
        x = self.dropout1(x)
        # x, _ = self.lstm4(x)
        x, _ = self.lstm5(x)
        x, _ = self.lstm6(x)
        x = self.dropout2(x)
        x, _ = self.lstm7(x)
        x = self.fc(x[:, -1,:])
        return x

In [28]:
# lstm모델 불러오기
model_path = './lstm_pth/lstm_model_03.pth'

lstm_model = hand_LSTM().to(device)
lstm_model.load_state_dict(torch.load(model_path, map_location=device))
lstm_model.eval()
print("모델이 성공적으로 불러와졌습니다.")

모델이 성공적으로 불러와졌습니다.


In [22]:
# mediapipe 손 감지 모듈 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.3)
mp_drawing = mp.solutions.drawing_utils

In [27]:
# YOLO 객체 감지 모델 초기화
best_model = 'block_best_02.pt'
yolo_model = YOLO(best_model)

In [24]:
# 객체와 손, dataset길이 설정
detect_cls = 0
hand_cls = 6
length = 10

In [29]:
cap_device = 0
cap = cv2.VideoCapture(cap_device) # for Mac
# cap = cv2.VideoCapture(0) # for Windows

lstm_model.eval()
status = 'None'
xyz_list_list = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # BGR 이미지를 RGB로 변환
    # image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    image = cv2.resize(frame, (640, 640))

    # YOLO 객체 감지
    box_results = yolo_model.predict(image, conf = 0.6)
    boxes = box_results[0].boxes.xyxy.cpu()
    box_class = box_results[0].boxes.cls.cpu().tolist()

    x1, y1, x2, y2 = 0, 0, 0, 0
    hx1, hy1, hx2, hy2 = 0,0,0,0
    for idx, cls in enumerate(box_class):
        if int(cls) == detect_cls:
            x1, y1, x2, y2 = boxes[idx]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        elif int(cls) == hand_cls:
            hx1, hy1, hx2, hy2 = boxes[idx]
            hx1, hy1, hx2, hy2 = int(hx1), int(hy1), int(hx2), int(hy2)
            cv2.rectangle(image, (int(hx1), int(hy1)), (int(hx2), int(hy2)), (0, 0, 255), 2)

    #mediapipe
    results = hands.process(image)
    xyz_list = []
    if results.multi_hand_landmarks:
        for x_y_z in results.multi_hand_landmarks:
            for landmark in x_y_z.landmark:
                xyz_list.append(landmark.x * 10)
                xyz_list.append(landmark.y * 10)
                xyz_list.append(landmark.z * 10)

        xyz_list.append(abs(x1-hx1))
        xyz_list.append(abs(x2-hx2))
        xyz_list.append(abs(y1-hy1))
        xyz_list.append(abs(y2-hy2))        
        # xyz_list.append(x1)
        # xyz_list.append(y1)
        # xyz_list.append(x2)
        # xyz_list.append(y2)
        # xyz_list.append(hx1)
        # xyz_list.append(hy1)
        # xyz_list.append(hx2)
        # xyz_list.append(hy2)

        if x1 != 0 and y1 != 0 and x2 != 0 and y2 != 0 and hx1 != 0 and hy1 != 0 and hx2 != 0 and hy2 != 0:
            xyz_list_list.append(xyz_list)

        for hand_landmarks in results.multi_hand_landmarks:
              with torch.no_grad():
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    

    if len(xyz_list_list) == length:
        dataset = []
        dataset.append({'key': 0, 'value': xyz_list_list})
        dataset = MyDataset(dataset)
        dataset = DataLoader(dataset)
        xyz_list_list = []
        for data, label in dataset:
            data = data.to(device)
            with torch.no_grad():
                result = lstm_model(data)
                _, out = torch.max(result, 1)
                if out.item() == 0: status = 'Release'
                else: status = 'Grab'

    cv2.putText(image, status, (0, 50), cv2.FONT_HERSHEY_COMPLEX, 1.5, (0,0, 225), 2)


    cv2.imshow('frame',image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 640x640 (no detections), 10.0ms
Speed: 4.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.0ms
Speed: 3.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 blue_bridge, 1 hand, 10.0ms
Speed: 3.0ms preprocess, 10.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 blue_bridge, 1 hand, 11.0ms
Speed: 3.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 blue_bridge, 1 hand, 10.0ms
Speed: 3.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 blue_bridge, 1 hand, 10.9ms
Speed: 2.0ms preprocess, 10.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 blue_bridge, 1 hand, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 blue_bridge, 1 hand, 9.0ms
Sp

In [15]:
cv2.destroyAllWindows()